# System Status

View the current status of:
- Active cycle
- Step progress
- Recent activity
- System health

In [1]:
# Load helpers
%run /workspace/system/helpers/03_StepTracker.ipynb
%run /workspace/system/helpers/04_Display.ipynb

✅ Configuration loaded - Base path: /workspace
✅ Database connection successful
✅ Step Tracker loaded
✅ Display utilities loaded


## Active Cycle Status

In [2]:
print_header("Active Cycle Status")

active_cycle = get_active_cycle()
if active_cycle:
    display_cycle_summary(active_cycle)
    
    # Get additional stats
    cycle_name = active_cycle['cycle_name']
    
    # Count completed steps
    query = """
    SELECT COUNT(DISTINCT st.id) as completed_steps
    FROM irp_step st
    INNER JOIN irp_stage sg ON st.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
    AND EXISTS (
        SELECT 1 FROM irp_step_run sr 
        WHERE sr.step_id = st.id AND sr.status = 'completed'
    )
    """
    completed = execute_scalar(query, (cycle_name,))
    
    # Total steps
    query = """
    SELECT COUNT(*) FROM irp_step st
    INNER JOIN irp_stage sg ON st.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
    """
    total = execute_scalar(query, (cycle_name,))
    
    print(f"\n📊 Progress Overview:")
    display_progress_bar(completed or 0, total or 0, title="Overall Completion")
    
else:
    print("⚠️  No active cycle")
    print("\nRun 'Create_New_Cycle.ipynb' to create a new cycle")

 Active Cycle Status
 Cycle: Q1_2024_Analysis
+------------+---------------------+
| Property   | Value               |
+============+=====================+
| Status     | active              |
+------------+---------------------+
| Created    | 2025-10-04 01:44:47 |
+------------+---------------------+
| Created By | notebook_user       |
+------------+---------------------+
| Stages     | 0                   |
+------------+---------------------+
| Steps      | 0                   |
+------------+---------------------+

📊 Progress Overview:
Overall Completion:
[░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/2)


## Step Progress Details

In [3]:
if active_cycle:
    print_header(f"Step Progress: {active_cycle['cycle_name']}")
    
    progress_df = get_current_progress(active_cycle['cycle_name'])
    if not progress_df.empty:
        display_step_progress(progress_df)
        
        # Summary statistics
        print("\n" + "=" * 60)
        print("Summary:")
        status_counts = progress_df['last_status'].value_counts(dropna=False)
        
        total_steps = len(progress_df)
        completed = status_counts.get('completed', 0)
        running = status_counts.get('running', 0)
        failed = status_counts.get('failed', 0)
        skipped = status_counts.get('skipped', 0)
        not_started = status_counts.get(None, 0) + status_counts.get(pd.NA, 0)
        
        summary_data = [
            ['Total Steps', total_steps],
            ['✅ Completed', completed],
            ['⏳ Running', running],
            ['❌ Failed', failed],
            ['⏭️ Skipped', skipped],
            ['⭕ Not Started', not_started]
        ]
        
        display_status_table(summary_data, headers=['Status', 'Count'])
    else:
        print("No steps registered for this cycle")

 Step Progress: Q1_2024_Analysis

📁 Stage 1: Setup
  --------------------------------------------------
  🔒 Step 1: Initialize
      Status: ⭕ Not Started
  🔒 Step 2: Validate
      Status: ⭕ Not Started

Summary:
+----------------+---------+
| Status         |   Count |
+================+=========+
| Total Steps    |       2 |
+----------------+---------+
| ✅ Completed   |       0 |
+----------------+---------+
| ⏳ Running     |       0 |
+----------------+---------+
| ❌ Failed      |       0 |
+----------------+---------+
| ⏭️ Skipped      |       0 |
+----------------+---------+
| ⭕ Not Started |       2 |
+----------------+---------+


## Recent Activity

In [4]:
if active_cycle:
    print_header("Recent Activity")
    
    history_df = get_step_history(active_cycle['cycle_name'])
    
    if not history_df.empty:
        # Show last 10 runs
        display_df = history_df[['stage_name', 'step_name', 'run_number', 'status', 
                                'started_ts', 'duration_seconds']].head(10)
        display_dataframe(display_df, title="Last 10 Step Runs")
    else:
        print("No step runs yet")

 Recent Activity
No step runs yet


## Directory Structure

In [5]:
if active_cycle:
    print_header("Active Cycle Directory")
    
    active_dir = WORKFLOWS_PATH / f"Active_{active_cycle['cycle_name']}"
    
    if active_dir.exists():
        print(f"📁 {active_dir}")
        
        def show_tree(path, prefix="  ", max_depth=3, current_depth=0):
            if current_depth >= max_depth:
                return
            
            items = sorted(path.iterdir())
            for item in items:
                if item.is_dir():
                    print(f"{prefix}📁 {item.name}/")
                    show_tree(item, prefix + "  ", max_depth, current_depth + 1)
                elif item.suffix == '.ipynb':
                    print(f"{prefix}📓 {item.name}")
                elif item.name not in ['.gitkeep', '.DS_Store']:
                    print(f"{prefix}📄 {item.name}")
        
        show_tree(active_dir)
    else:
        print(f"❌ Directory not found: {active_dir}")

 Active Cycle Directory
📁 /workspace/workflows/Active_Q1_2024_Analysis
  📄 README.md
  📁 files/
  📁 logs/
  📁 notebooks/
    📁 Stage_01_Setup/
      📁 .ipynb_checkpoints/
      📄 README.md
      📓 Step_01_Initialize.ipynb
      📓 Step_02_Validate.ipynb


## System Health Check

In [7]:
print_header("System Health")

health_checks = []

# Database connection
try:
    if test_connection():
        health_checks.append(['Database', '✅ Connected'])
    else:
        health_checks.append(['Database', '❌ Not Connected'])
except:
    health_checks.append(['Database', '❌ Error'])

# Template directory
if TEMPLATE_PATH.exists():
    health_checks.append(['Template', '✅ Found'])
else:
    health_checks.append(['Template', '⚠️ Missing'])

# Active cycle
if active_cycle:
    health_checks.append(['Active Cycle', f"✅ {active_cycle['cycle_name']}"])
else:
    health_checks.append(['Active Cycle', '⚠️ None'])

# System lock
lock = execute_query("SELECT * FROM irp_system_lock WHERE id = 1")
if not lock.empty:
    health_checks.append(['System Lock', '✅ Initialized'])
else:
    health_checks.append(['System Lock', '❌ Not Initialized'])

display_status_table(health_checks, headers=['Component', 'Status'])

print("\n💡 Quick Actions:")
print("  • Create new cycle: Run 'Create_New_Cycle.ipynb'")
print("  • Initialize database: Run 'Database_Admin.ipynb'")
print("  • View step details: Check notebooks in Active_* directory")

 System Health
+--------------+---------------------+
| Component    | Status              |
+==============+=====================+
| Database     | ✅ Connected        |
+--------------+---------------------+
| Template     | ✅ Found            |
+--------------+---------------------+
| Active Cycle | ✅ Q1_2024_Analysis |
+--------------+---------------------+
| System Lock  | ✅ Initialized      |
+--------------+---------------------+

💡 Quick Actions:
  • Create new cycle: Run 'Create_New_Cycle.ipynb'
  • Initialize database: Run 'Database_Admin.ipynb'
  • View step details: Check notebooks in Active_* directory


---